# Import necessary depencencies

In [ ]:
import numpy as np
from keras.layers import Dropout, Activation, Dense
from keras.models import Sequential
import model_evaluation_utils as meu
import utils
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM
import xgboost as xgb
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict, validation_curve
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import pandas as pd
from keras.regularizers import l2
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version
#from scipy.misc import comb
from scipy.special import comb
import math
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import sklearn
import utils
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import cross_val_score
else:
    from sklearn.model_selection import cross_val_score

np.set_printoptions(precision=2, linewidth=80)
NWORKERS=16

# Load normalized data

In [ ]:
# train and test datasets
train_reviews = utils.readFromDisk('train_reviews')
train_sentiments = utils.readFromDisk('train_sentiments')
test_reviews = utils.readFromDisk('test_reviews')
test_sentiments = utils.readFromDisk('test_sentiments')
tokenized_train = utils.readFromDisk('tokenized_train')
tokenized_test = utils.readFromDisk('tokenized_test')
train_sentiments_encoded = utils.readFromDisk('train_sentiments_encoded')
test_sentiments_encoded = utils.readFromDisk('test_sentiments_encoded')


train_features = utils.readFromDisk('train_ft_features')
test_features = utils.readFromDisk('test_ft_features')

#labelencoder used for encoding in feature engg
le = utils.readFromDisk('label_encoder')


In [ ]:
#split train into train and vaildate sets for models that need it
def splitTrainForValidation(train_x,train_y,val_ratio):
    X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=val_ratio)
    return X_train, X_val, y_train, y_val


# Logistic Regression (hyperparameter tuning and caching)


In [ ]:
#validation curves
param_name = 'C'
param_range=np.linspace(1, 15, 4,dtype=int)
classifier=LogisticRegression()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)


In [ ]:
try:
    lr=utils.readFromDisk('lr')
except FileNotFoundError as te:
    lr0 = LogisticRegression()
    lr_hyperparameters = {    
        'C' : [5],
        #'penalty' : ['l2'],
        'max_iter':[1000]
    }
    lr_clf = RandomizedSearchCV(lr0, lr_hyperparameters,cv=5, n_iter=10,n_jobs=-1)

    # Fit grid search
    lr = lr_clf.fit(train_features,train_sentiments)

    utils.writeToDisk(lr,'lr')

In [ ]:
    # View best hyperparameters
    lr.best_estimator_.get_params()

In [ ]:
# Predict target vector
lr_predictions=lr.predict(test_features)


## Logistic Regression - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=lr_predictions,
                                      classes=[1,0])


## Logistic Regression - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,lr.best_estimator_,5,np.linspace(.1, 0.7, 7))

# SVM using sgd classifier (hyperparameter tuning and caching)


In [ ]:
#validation curves
param_name = 'alpha'
param_range=[0.00001,0.0001, 0.001, 0.01, 0.1]
classifier=SGDClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)

In [ ]:
try:
    svm=utils.readFromDisk('svm')
except FileNotFoundError as te:
    #svm0 = SGDClassifier(max_iter=500,n_jobs=NWORKERS)
    svm0 = SGDClassifier()

    svm_n_jobs=[NWORKERS]
    svm_max_iter=[1000]
    svm_loss= ["hinge"]
    svm_alpha = [0.00001]
    svm_penalty = ["l2"]
    svm_hyperparameters = dict(max_iter=[1000],loss=svm_loss, alpha=svm_alpha,penalty=svm_penalty,n_jobs=svm_n_jobs)

    svm_clf = GridSearchCV(svm0, svm_hyperparameters,cv=5, n_jobs=1)

    svm = svm_clf.fit(train_features,train_sentiments)

    utils.writeToDisk(svm,'svm')


In [ ]:
    svm.best_estimator_.get_params()

In [ ]:
svm_predictions=svm.predict(test_features)

## SVM - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=svm_predictions,
                                      classes=[1,0])


## SVM - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,svm.best_estimator_,5,np.linspace(.1, 0.7, 7))



# Naive Bayes (hyperparameter tuning and caching)


In [ ]:
try:
    nb=utils.readFromDisk('nb')
except FileNotFoundError as te:
    nb0 = GaussianNB()

    nb_hyperparameters = {
        #'priors': np.linspace(0.1, 0.9, 6)
       #'fit_prior': [True, False],  
    }
    nb_clf = GridSearchCV(nb0,nb_hyperparameters, cv=5, n_jobs=-1)

    nb = nb_clf.fit(train_features,train_sentiments)

    utils.writeToDisk(nb,'nb')


In [ ]:
    nb.best_estimator_.get_params()

In [ ]:
nb_predictions=nb.predict(test_features)

## Naive Bayes  - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=nb_predictions,
                                      classes=[1,0])


## Naive Bayes - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,nb.best_estimator_,5,np.linspace(.1, 0.7, 7))



# Random Forest (hyperparameter tuning and caching)


In [ ]:
#validation curves
param_name = 'n_estimators'
param_range=[10, 50, 100, 150, 200]
classifier=RandomForestClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)



In [ ]:
#validation curves
param_name = 'min_samples_leaf'
param_range=[1,3,5]
classifier=RandomForestClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)

In [ ]:
#validation curves

param_name = 'max_depth'
param_range=[200, 500, 1000]
classifier=RandomForestClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)

In [ ]:
#validation curves
param_name = 'min_samples_split'
param_range=[2,3,4]
classifier=RandomForestClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)

In [ ]:
#hyper parameter tuning
try:
    rf=utils.readFromDisk('rf')
except FileNotFoundError as te:
    rf0 = RandomForestClassifier()

    rf_hyperparameters = {
        'n_estimators': [50],
        'max_features': ['auto'],
        'min_samples_leaf': [1],
        'random_state':[1],
        'oob_score': [True],
        'max_depth':[200],
        'min_samples_split':[2],
        'n_jobs':[NWORKERS]
    }
    rf_clf = RandomizedSearchCV(rf0,rf_hyperparameters, cv=5, n_jobs=NWORKERS)

    rf = rf_clf.fit(train_features,train_sentiments)

    utils.writeToDisk(rf,'rf')


In [ ]:
    rf.best_estimator_.get_params()

In [ ]:
rf_predictions=rf.predict(test_features)

## Random Forest  - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=rf_predictions,
                                      classes=[1,0])


## Random Forest - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,rf.best_estimator_,5,np.linspace(.1, 0.7, 7))



# KNN (hyperparameter tuned)

In [ ]:
#validation curves
param_name = 'n_neighbors'
param_range=[1,2,3,4,5]
classifier=KNeighborsClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)


In [ ]:
try:
    knn=utils.readFromDisk('knn')
except FileNotFoundError as te:
    knn0 = KNeighborsClassifier()

    knn_hyperparameters = {
        'n_neighbors': [1]
    }
    knn_clf = GridSearchCV(knn0,knn_hyperparameters, cv=5, n_jobs=-1)

    knn = knn_clf.fit(train_features,train_sentiments)

    utils.writeToDisk(knn,'knn')


In [ ]:
    knn.best_estimator_.get_params()

In [ ]:
knn_predictions=knn.predict(test_features)

## KNN  - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=knn_predictions,
                                      classes=[1,0])



## KNN - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,knn.best_estimator_,5,np.linspace(.1, 0.7, 7))



# XGBoost (hyperparameter tuning and caching)


Parameters
max_depth (int) – Maximum tree depth for base learners.

learning_rate (float) – Boosting learning rate (xgb’s “eta”)

n_estimators (int) – Number of trees to fit.

verbosity (int) – The degree of verbosity. Valid values are 0 (silent) - 3 (debug).

objective (string or callable) – Specify the learning task and the corresponding learning objective or a custom objective function to be used (see note below).

booster (string) – Specify which booster to use: gbtree, gblinear or dart.

tree_method (string) – Specify which tree method to use. Default to auto. If this parameter is set to default, XGBoost will choose the most conservative option available. It’s recommended to study this option from parameters document.

n_jobs (int) – Number of parallel threads used to run xgboost.

gamma (float) – Minimum loss reduction required to make a further partition on a leaf node of the tree.

min_child_weight (int) – Minimum sum of instance weight(hessian) needed in a child.

max_delta_step (int) – Maximum delta step we allow each tree’s weight estimation to be.

subsample (float) – Subsample ratio of the training instance.

colsample_bytree (float) – Subsample ratio of columns when constructing each tree.

colsample_bylevel (float) – Subsample ratio of columns for each level.

colsample_bynode (float) – Subsample ratio of columns for each split.

reg_alpha (float (xgb's alpha)) – L1 regularization term on weights

reg_lambda (float (xgb's lambda)) – L2 regularization term on weights

scale_pos_weight (float) – Balancing of positive and negative weights.

base_score – The initial prediction score of all instances, global bias.

random_state (int) –

Random number seed.

Note

Using gblinear booster with shotgun updater is nondeterministic as it uses Hogwild algorithm.

missing (float, optional) – Value in the data which needs to be present as a missing value. If None, defaults to np.nan.

num_parallel_tree (int) – Used for boosting random forest.

importance_type (string, default "gain") – The feature importance type for the feature_importances_ property: either “gain”, “weight”, “cover”, “total_gain” or “total_cover”.

In [ ]:
#validation curves
param_name = 'max_depth'
param_range=np.linspace(10, 200, 5)
classifier=xgb.XGBClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)




In [ ]:
#validation curves
param_name = 'n_estimators'
param_range=np.linspace(10, 50, 100)
classifier=xgb.XGBClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)



In [ ]:
#validation curves
param_name = 'learning_rate'
param_range=[0.01, 0.05,0.1]
classifier=xgb.XGBClassifier()
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)




In [ ]:
try:
    xg=utils.readFromDisk('xg')
except FileNotFoundError as te:
#    xg = xgb.XGBClassifier(n_estimators=500, max_depth=5, base_score=0.5,objective='binary:logistic', 
#                       random_state=42,nthread=16,n_jobs=NWORKERS)
    xg0 = xgb.XGBClassifier()

    xg_hyperparameters = {
        'max_depth': [200],
        'n_estimators': [50],
        'learning_rate': [0.01],
        'objective': ['binary:logistic'],
        'random_state': [42],
        'n_jobs': [NWORKERS]
    }

    xg_clf = GridSearchCV(xg0, xg_hyperparameters,cv=5, n_jobs=1)
    
    xg=xg_clf.fit(train_features, train_sentiments)

    utils.writeToDisk(xg,'xg')


In [ ]:
    xg.best_estimator_.get_params()

In [ ]:
xg_predictions= xg.predict(test_features)


## XGBoost - Performance


In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=xg_predictions, 
                                      classes=[1,0])  


## XGBoost - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,xg.best_estimator_,5,np.linspace(.1, 0.7, 7))



# Deep neural network architecture (DNN) (hyperparameter tuning and caching)


In [ ]:
#validation curves
param_name = ''
param_range=[]
classifier=
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)


In [ ]:
def create_dnn_model(optimizer='adam', init='glorot_uniform'):    
    dnn_model = Sequential()
    dnn_model.add(Dense(512, activation='relu', input_shape=(300,)))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(512, activation='relu'))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(512, activation='relu'))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(2))
    dnn_model.add(Activation('softmax'))

    dnn_model.compile(loss='categorical_crossentropy', optimizer=optimizer,                 
                      metrics=['accuracy'])
    return dnn_model


## DNN - Training and Prediction


In [ ]:

try:   
    dnn=utils.readFromDisk('dnn')
except FileNotFoundError as te:
    #see https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/
    dnn0 = KerasClassifier(build_fn = create_dnn_model)

    # grid search epochs, batch size and optimizer
    #dnn_optimizers = ['rmsprop', 'adam']
    #dnn_init = ['glorot_uniform', 'normal', 'uniform']
    dnn_epochs = [10,25,50]
    dnn_batch_size=[50,100,200,300]
    dnn_validation_split=[0.1]
    dnn_shuffle=[True]
    dnn_workers=[NWORKERS]
    dnn_use_multiprocessing=[True]
    dnn_verbose=[1]
    dnn_hyperparameters = dict(epochs=dnn_epochs, 
                               batch_size=dnn_batch_size,validation_split=dnn_validation_split,
                               shuffle=dnn_shuffle, workers=dnn_workers, use_multiprocessing=dnn_use_multiprocessing,
                               verbose=dnn_verbose)
    
    
    dnn_clf = GridSearchCV(dnn0, dnn_hyperparameters,cv=5, n_jobs=1)
   
    dnn=dnn_clf.fit(train_features,train_sentiments_encoded)    
    
    utils.writeToDisk(dnn,'dnn')


In [ ]:
    dnn.best_estimator_.get_params()


In [ ]:
y_pred = dnn.predict(test_features)
dnn_predictions = le.inverse_transform(y_pred) 

## DNN - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=dnn_predictions, 
                                      classes=[1,0])  


## DNN - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,dnn.best_estimator_,5,np.linspace(.1, 0.7, 7))




# LSTM


In [ ]:
#validation curves
param_name = ''
param_range=[]
classifier=
title=param_name

meu.validationCurve(classifier,train_features,train_sentiments,param_name,param_range,NWORKERS,title)


In [ ]:

# build word to index vocabulary
token_counter = Counter([token for review in tokenized_train for token in review])
vocab_map = {item[0]: index+1 for index, item in enumerate(dict(token_counter).items())}
max_index = np.max(list(vocab_map.values()))
vocab_map['PAD_INDEX'] = 0
vocab_map['NOT_FOUND_INDEX'] = max_index+1
vocab_size = len(vocab_map)
# view vocabulary size and part of the vocabulary map
print('Vocabulary Size:', vocab_size)
print('Sample slice of vocabulary map:', dict(list(vocab_map.items())[10:20]))

## LSTM- Encode and Pad datasets & Encode prediction class labels

In [ ]:
# get max length of train corpus and initialize label encoder
max_len = np.max([len(review) for review in tokenized_train])

## Train reviews data corpus
# Convert tokenized text reviews to numeric vectors
train_X = [[vocab_map[token] for token in tokenized_review] for tokenized_review in tokenized_train]
train_X = sequence.pad_sequences(train_X, maxlen=max_len) # pad 

## Test reviews data corpus
# Convert tokenized text reviews to numeric vectors
test_X = [[vocab_map[token] if vocab_map.get(token) else vocab_map['NOT_FOUND_INDEX'] 
           for token in tokenized_review] 
              for tokenized_review in tokenized_test]
test_X = sequence.pad_sequences(test_X, maxlen=max_len)

# view vector shapes
print('Max length of train review vectors:', max_len)
print('Train review vectors shape:', train_X.shape, ' Test review vectors shape:', test_X.shape)


In [ ]:
EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

def create_lstm_model(optimizer='adam', init='glorot_uniform'):    
    lstm = Sequential()

    lstm.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len))
    lstm.add(SpatialDropout1D(0.2))
    lstm.add(LSTM(LSTM_DIM, dropout=0.2, recurrent_dropout=0.2))
    lstm.add(Dense(2, activation="sigmoid"))
    
    lstm.compile(loss="binary_crossentropy", optimizer="adam",
                  metrics=["accuracy"])

    return lstm




## LSTM- Build, train and visualize the LSTM Model

In [ ]:
try:   
    lstm=utils.readFromDisk('lstm')
    print(lstm.summary())
except FileNotFoundError as te:    
    
    #see https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/
    lstm0 = KerasClassifier(build_fn = create_lstm_model)

    # grid search epochs, batch size and optimizer
    #dnn_optimizers = ['rmsprop', 'adam']
    #dnn_init = ['glorot_uniform', 'normal', 'uniform']
    lstm_epochs = [1]
    lstm_batch_size=[100]
    lstm_validation_split=[0.1]
    lstm_shuffle=[True]
    lstm_workers=[NWORKERS]
    lstm_use_multiprocessing=[True]
    lstm_verbose=[1]
    lstm_hyperparameters = dict(epochs=lstm_epochs, 
                               batch_size=lstm_batch_size,validation_split=lstm_validation_split,
                               shuffle=lstm_shuffle, workers=lstm_workers, use_multiprocessing=lstm_use_multiprocessing,
                               verbose=lstm_verbose)
    
    
    lstm_clf = GridSearchCV(lstm0, lstm_hyperparameters,cv=5, n_jobs=1)
   
    lstm=lstm_clf.fit(train_X, train_sentiments_encoded)    
    
    #lstm=create_lstm_model()
    #lstm.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len))
    #lstm.add(SpatialDropout1D(0.2))
    #lstm.add(LSTM(LSTM_DIM, dropout=0.2, recurrent_dropout=0.2))
    #lstm.add(Dense(2, activation="sigmoid"))
    #lstm.compile(loss="binary_crossentropy", optimizer="adam",
    #              metrics=["accuracy"])
    #print(lstm.summary())

    #SVG(model_to_dot(lstm, show_shapes=True, show_layer_names=False, 
    #                 rankdir='LR').create(prog='dot', format='svg'))
    #batch_size = 100
    #lstm.fit(train_X, train_sentiments_encoded, epochs=2, batch_size=batch_size, 
    #          shuffle=True, validation_split=0.1, verbose=1,workers=NWORKERS,use_multiprocessing=True)
    utils.writeToDisk(lstm,'lstm')

In [ ]:
lstm.best_estimator_.get_params()



## LSTM - Predictions

In [ ]:
lstm_pred_test = lstm.predict(test_X)

In [ ]:
lstm_predictions = le.inverse_transform(lstm_pred_test)

## LSTM - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=lstm_predictions, 
                                      classes=[1,0])  



## LSTM - Learning Curve

In [ ]:
meu.learningCurve(train_features,train_sentiments,lstm.best_estimator_,5,np.linspace(.1, 0.7, 7))



# Ensemble Stacking


In [ ]:
stack_clf1 = KNeighborsClassifier(n_neighbors=1,n_jobs=3)
stack_clf2 = RandomForestClassifier(random_state=1,n_jobs=3)
stack_clf3 = GaussianNB()
stack_lr = LogisticRegression(penalty='l2', max_iter=500, C=60,n_jobs=3)
sclf = StackingClassifier(classifiers=[stack_clf1, stack_clf2, stack_clf3], 
                          meta_classifier=stack_lr)

## Ensemble Stacking - Predictions via n fold validations

In [ ]:
stack_clf_list = [stack_clf1, stack_clf2, stack_clf3, sclf]
#preds_list=[]

try:
    stack_clf_list=utils.readFromDisk('stack_clf_list')
    #preds_list=utils.readFromDisk('preds_list')
except FileNotFoundError as te:   
    stack_clf_list = [stack_clf1, stack_clf2, stack_clf3, sclf]
    #stack_preds_list=[]
    for stack_clf in stack_clf_list:
        stack_clf=stack_clf.fit(train_features,train_sentiments)
        #pred = cross_val_predict(clf, train_features, train_sentiments, cv=2,n_jobs=NWORKERS)
        #preds_list.append(pred)
    
    utils.writeToDisk(stack_clf_list,'stack_clf_list')
    #utils.writeToDisk(preds_list,'preds_list')

In [ ]:
try:
    stack_knn_predictions=utils.readFromDisk('stack_knn_predictions')
except FileNotFoundError as te:    
    stack_knn_predictions=stack_clf1.predict(test_features)
    utils.writeToDisk(stack_knn_predictions,'stack_knn_predictions')
try:
    stack_rf_predictions=utils.readFromDisk('stack_rf_predictions')
except FileNotFoundError as te:    
    stack_rf_predictions=stack_clf2.predict(test_features)
    utils.writeToDisk(stack_rf_predictions,'stack_rf_predictions')

try:
    stack_gau_predictions=utils.readFromDisk('stack_gau_predictions')
except FileNotFoundError as te:    
    stack_gau_predictions=stack_clf3.predict(test_features)
    utils.writeToDisk(stack_gau_predictions,'stack_gau_predictions')

try:
    stack_stack_predictions=utils.readFromDisk('stack_predictions')
except FileNotFoundError as te:    
    stack_predictions=sclf.predict(test_features)
    utils.writeToDisk(stack_predictions,'stack_predictions')
    

## Ensemble Stacking Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=stack_knn_predictions, 
                                      classes=[1,0])  

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=stack_rf_predictions, 
                                      classes=[1,0])  

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=stack_gau_predictions, 
                                      classes=[1,0])  

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=stack_predictions, 
                                      classes=[1,0])  

# Ensemble Majority Vote


In [ ]:
class MajorityVoteClassifier(BaseEstimator, 
                             ClassifierMixin):
    """ A majority vote ensemble classifier

    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble

    vote : str, {'classlabel', 'probability'} (default='label')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).

    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.

    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.

        y : array-like, shape = [n_samples]
            Vector of target class labels.

        Returns
        -------
        self : object

        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.

        Returns
        ----------
        maj_vote : array-like, shape = [n_samples]
            Predicted class labels.
            
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        avg_proba : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.

        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
            return out


In [ ]:
maj_clf1 = LogisticRegression(penalty='l2', C=0.1, max_iter=500, random_state=0,n_jobs=3)
maj_clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=0)
maj_clf3 = KNeighborsClassifier(n_neighbors=1,p=2, metric='minkowski',n_jobs=3)
maj_pipe1 = Pipeline([['sc', StandardScaler()],
                  ['clf', maj_clf1]])
maj_pipe3 = Pipeline([['sc', StandardScaler()],
                  ['clf', maj_clf3]])
maj_clf_labels = ['Logistic Regression', 'Decision Tree', 'KNN','Majority Voting']
mclf = MajorityVoteClassifier(classifiers=[maj_pipe1, maj_clf2, maj_pipe3])
maj_clf_list = [maj_pipe1, maj_clf2, maj_pipe3, mclf]


try:
    maj_clf_list=utils.readFromDisk('maj_clf_list')
except FileNotFoundError as te:   
    maj_clf_list = [maj_pipe1, maj_clf2, maj_pipe3, mclf]
    for maj_clf in maj_clf_list:
        maj_clf=maj_clf.fit(train_features,train_sentiments)
    
    utils.writeToDisk(maj_clf_list,'maj_clf_list')

In [ ]:
try:
    maj_lr_predictions=utils.readFromDisk('maj_lr_predictions')
except FileNotFoundError as te:    
    maj_lr_predictions=maj_clf1.predict(test_features)
    utils.writeToDisk(maj_lr_predictions,'maj_lr_predictions')
try:
    maj_dt_predictions=utils.readFromDisk('maj_dt_predictions')
except FileNotFoundError as te:    
    maj_dt_predictions=maj_clf2.predict(test_features)
    utils.writeToDisk(maj_dt_predictions,'maj_dt_predictions')

try:
    maj_knn_predictions=utils.readFromDisk('maj_knn_predictions')
except FileNotFoundError as te:    
    maj_knn_predictions=maj_clf3.predict(test_features)
    utils.writeToDisk(maj_knn_predictions,'maj_knn_predictions')


try:
    maj_predictions=utils.readFromDisk('maj_predictions')
except FileNotFoundError as te:    
    maj_predictions=mclf.predict(test_features)
    utils.writeToDisk(maj_predictions,'maj_predictions')
    

## Ensemble Majority Voting Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=maj_lr_predictions,
                                      classes=[1,0])  

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=maj_dt_predictions, 
                                      classes=[1,0])  

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=maj_knn_predictions, 
                                      classes=[1,0])  

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=maj_predictions, 
                                      classes=[1,0])  



# All Models Evaluation


## Model ROC curves

In [ ]:
plt.figure(0).clf()

color = ['blue', 'orange', 'red', 'green', 'coral',
             'grey', 'indigo', 'gold', 'lime', 'olive',
             'pink', 'navy', 'magenta', 'yellow', 'tomato',
             'turquoise', 'yellowgreen', 'maroon', 'lightblue']
mlr=[]
msvm=[]
mdnn=[]
mxg=[]
mlstm=[]
mknn=[]
mrf=[]
mgau=[]
mstack=[]

def metricsAndROC(pred,metricsArray,rocTitle,colorIndex):
    fpr, tpr, thresholds = metrics.roc_curve(test_sentiments, pred)
    auc = metrics.roc_auc_score(test_sentiments, pred)
    metricsArray.append(metrics.f1_score(test_sentiments, pred))
    metricsArray.append(metrics.precision_score(test_sentiments, pred))
    metricsArray.append(metrics.accuracy_score(test_sentiments, pred))
    metricsArray.append(metrics.recall_score(test_sentiments, pred))
    
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr,color=color[colorIndex], label=rocTitle.format(auc))



metricsAndROC(lr_predictions,mlr,'LR',0)
metricsAndROC(svm_predictions,msvm,'SVM',1)
metricsAndROC(dnn_predictions,mdnn,'DNN',2)
metricsAndROC(xg_predictions,mxg,'XGBoost',3)
metricsAndROC(lstm_predictions,mlstm,'LSTM',4)
metricsAndROC(stack_knn_predictions,mknn,'KNN',5)
metricsAndROC(stack_rf_predictions,mrf,'RFOR',6)
metricsAndROC(stack_gau_predictions,mgau,'GAUS',7)
metricsAndROC(stack_predictions,mstack,'ENSEM-STACK',8)



#show the roc curve now
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
# show the legend
plt.legend(loc='center right')
plt.show()

## All Models Metrics comparison

In [ ]:
n_groups = 4
index = np.arange(n_groups)
bar_width = .1

plt.bar(index,mlr, bar_width, color=color[0], label='LR')


z=index + bar_width
plt.bar(z, msvm, bar_width, color=color[1],label='SVM')


z=z+ bar_width
plt.bar(z, mdnn, bar_width, color=color[2], label='DNN')

z=z+ bar_width
plt.bar(z,mxg , bar_width, color=color[3], label='XGB')

z=z+ bar_width
plt.bar(z,mlstm , bar_width,color=color[4], label='LSTM')



#ax.set_xlabel('Metric')
#ax.set_ylabel('Value')
#ax.set_title('Comparison of Feature Engineering Models on Amazon Reviews')
#ax.set_xticks(index + bar_width / 2)
pltLabels=['F1','PRECISION','ACCURACY','RECALL']
# Add xticks on the middle of the group bars
plt.xlabel('Model Metrics', fontweight='bold')
plt.xticks([r + bar_width for r in range(n_groups)], pltLabels)
 
# Create legend & Show graphic
plt.legend(frameon=False,ncol=3, loc='lower left')
plt.show()
